## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [81]:
lng = 'fr'
acteur = 'inesss'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [82]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [83]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        csv.insert(1, "Corpus", acteur)
        csv.insert(2, "Sous-corpus", tag)
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'text']]
    return df

In [84]:
data = lire_corpus(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 2035 documents.


In [85]:
base_path = '../03-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Corpus,Sous-corpus,Title,text
0,inesss,biologie-medicale-et-genomique,INESSS: Biologie médicale et génomique,Biologie médicale et génomique Processus et cr...
1,inesss,biologie-medicale-et-genomique,INESSS: Processus et critères d'évaluation,Biologie médicale et génomique Processus et cr...
2,inesss,biologie-medicale-et-genomique,INESSS: Processus synchronisé d'évaluation des...,Biologie médicale et génomique Processus et cr...
3,inesss,biologie-medicale-et-genomique,<span>Soutien au diagnostic moléculaire<//span>,Biologie médicale et génomique Processus et cr...
4,inesss,biologie-medicale-et-genomique,<span>Avis au ministre<//span>,Biologie médicale et génomique Processus et cr...
...,...,...,...,...
26,inesss,traumatologie,5,Accueil Continuum de services en traumatologie...
27,inesss,traumatologie,Lire plus,Sondage - Qu’avez-vous pensé de ce guide : Ind...
28,inesss,traumatologie,Lire plus,Notice Santécom: 127161 Sondage – Qu’avez-vous...
29,inesss,traumatologie,"Les caractéristiques, l’historique et l’implan...",Le continuum de services en traumatologie (CST...


### **Nettoyage**

In [86]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [87]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 2035 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [88]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [89]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [90]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 1008425 tokens.
Le POS tagging devrait prendre environ 1 minutes.


In [91]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append((t.split('\t')[0], t.split('\t')[1]))
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [92]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [93]:
tagged

[('eprex', 'NOM'),
 ('dénomination', 'NOM'),
 ('commune', 'ADJ'),
 ('STOP', 'NOM'),
 ('STOP', 'NOM'),
 ('sujet', 'ADJ'),
 (':', 'PUN'),
 ('époétine', 'ADJ'),
 ('alfa', 'NOM'),
 ('nom', 'NOM'),
 ('du', 'PRP:det'),
 ('fabricant', 'NOM'),
 (':', 'PUN'),
 ('j.o.i.', 'ABR'),
 ('forme', 'VER:pres'),
 (':', 'PUN'),
 ('STOP', 'INT'),
 ('teneur', 'NOM'),
 (':', 'PUN'),
 ('STOP', 'NOM'),
 ('indication', 'NOM'),
 (':', 'PUN'),
 ('anémie', 'VER:pres'),
 ('recommandation', 'NOM'),
 ('de', 'PRP'),
 ("l'", 'DET:ART'),
 ('inesss', 'ADJ'),
 ('modification', 'NOM'),
 ("d'", 'PRP'),
 ('une', 'DET:ART'),
 ('indication', 'NOM'),
 ('reconnue', 'VER:pper'),
 ('par', 'PRP'),
 ('le', 'DET:ART'),
 ('conseil', 'NOM'),
 ('STOP', 'INT'),
 ('médicament', 'NOM'),
 ("d'", 'PRP'),
 ('exception', 'NOM'),
 ('décision', 'NOM'),
 ('du', 'PRP:det'),
 ('ministre', 'NOM'),
 ('information', 'NOM'),
 ('actuellement', 'ADV'),
 ('non', 'ADV'),
 ('disponible', 'ADJ'),
 ('en', 'PRP'),
 ('ligne', 'NOM'),
 ('evaluation', 'NOM'),
 ('

In [94]:
# Si on veut lemmatiser
### **Mapping POS Tags** (FRMG)

#Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [95]:
#tagged_lem = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]

In [96]:
# Si on veut lemmatiser
### **Lemmatisation** (FrenchLefffLemmatizer)
#https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

# lemmas = []
# for term in tagged_lem:
#     term_l = []
#     if lemmatizer.lemmatize(term[0], term[1]) == []:
#        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
#     elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
#         term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

#     else:
#         term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])

#     lemmas.append(term_l)

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [97]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=1, max_len=10))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [98]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 10796435 ngrammes.


In [99]:
ngrammes

[(('eprex', 'NOM'),),
 (('eprex', 'NOM'), ('dénomination', 'NOM')),
 (('eprex', 'NOM'), ('dénomination', 'NOM'), ('commune', 'ADJ')),
 (('eprex', 'NOM'),
  ('dénomination', 'NOM'),
  ('commune', 'ADJ'),
  ('STOP', 'NOM')),
 (('eprex', 'NOM'),
  ('dénomination', 'NOM'),
  ('commune', 'ADJ'),
  ('STOP', 'NOM'),
  ('STOP', 'NOM')),
 (('eprex', 'NOM'),
  ('dénomination', 'NOM'),
  ('commune', 'ADJ'),
  ('STOP', 'NOM'),
  ('STOP', 'NOM'),
  ('sujet', 'ADJ')),
 (('eprex', 'NOM'),
  ('dénomination', 'NOM'),
  ('commune', 'ADJ'),
  ('STOP', 'NOM'),
  ('STOP', 'NOM'),
  ('sujet', 'ADJ'),
  (':', 'PUN')),
 (('eprex', 'NOM'),
  ('dénomination', 'NOM'),
  ('commune', 'ADJ'),
  ('STOP', 'NOM'),
  ('STOP', 'NOM'),
  ('sujet', 'ADJ'),
  (':', 'PUN'),
  ('époétine', 'ADJ')),
 (('eprex', 'NOM'),
  ('dénomination', 'NOM'),
  ('commune', 'ADJ'),
  ('STOP', 'NOM'),
  ('STOP', 'NOM'),
  ('sujet', 'ADJ'),
  (':', 'PUN'),
  ('époétine', 'ADJ'),
  ('alfa', 'NOM')),
 (('eprex', 'NOM'),
  ('dénomination', 'NOM'

### **Extraction des patrons syntaxiques**

In [100]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [101]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [102]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [103]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [104]:
frequencies

FreqDist({('STOP',): 59180, ('de',): 52152, (')',): 27611, ('(',): 27524, ('la',): 21106, ("d'",): 20794, ('.',): 19498, ("l'",): 19334, ('des',): 18295, ('et',): 17645, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [105]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [106]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [107]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [108]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

In [109]:
phrases = filter_len(phrases)

In [110]:
phrases

[[('eprex',), ['NOM']],
 [('eprex', 'dénomination'), ['NOM', 'NOM']],
 [('eprex', 'dénomination', 'commune'), ['NOM', 'NOM', 'ADJ']],
 [('dénomination',), ['NOM']],
 [('dénomination', 'commune'), ['NOM', 'ADJ']],
 [('alfa',), ['NOM']],
 [('alfa', 'nom'), ['NOM', 'NOM']],
 [('alfa', 'nom', 'du', 'fabricant'), ['NOM', 'NOM', 'PRP:det', 'NOM']],
 [('nom',), ['NOM']],
 [('nom', 'du', 'fabricant'), ['NOM', 'PRP:det', 'NOM']],
 [('fabricant',), ['NOM']],
 [('teneur',), ['NOM']],
 [('indication',), ['NOM']],
 [('recommandation',), ['NOM']],
 [('recommandation', 'de', "l'", 'inesss'), ['NOM', 'PRP', 'DET:ART', 'ADJ']],
 [('recommandation', 'de', "l'", 'inesss', 'modification'),
  ['NOM', 'PRP', 'DET:ART', 'ADJ', 'NOM']],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indication'),
  ['NOM', 'PRP', 'DET:ART', 'ADJ', 'NOM', 'PRP', 'DET:ART', 'NOM']],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indica

On retire les n-grammes qui apparaissent moins de 15 fois dans le corpus

In [111]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 15]

In [112]:
phrases = filter_freq(phrases)

In [113]:
phrases

[[('dénomination',), ['NOM']],
 [('dénomination', 'commune'), ['NOM', 'ADJ']],
 [('alfa',), ['NOM']],
 [('nom',), ['NOM']],
 [('nom', 'du', 'fabricant'), ['NOM', 'PRP:det', 'NOM']],
 [('fabricant',), ['NOM']],
 [('teneur',), ['NOM']],
 [('indication',), ['NOM']],
 [('recommandation',), ['NOM']],
 [('recommandation', 'de', "l'", 'inesss'), ['NOM', 'PRP', 'DET:ART', 'ADJ']],
 [('recommandation', 'de', "l'", 'inesss', 'modification'),
  ['NOM', 'PRP', 'DET:ART', 'ADJ', 'NOM']],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indication'),
  ['NOM', 'PRP', 'DET:ART', 'ADJ', 'NOM', 'PRP', 'DET:ART', 'NOM']],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indication',
   'reconnue'),
  ['NOM',
   'PRP',
   'DET:ART',
   'ADJ',
   'NOM',
   'PRP',
   'DET:ART',
   'NOM',
   'VER:pper']],
 [('modification',), ['NOM']],
 [('modification', "d'", 'une', 'indication'),
  ['NOM', 'PRP', 'DET:ART', 'NOM']],


In [114]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [115]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [116]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 672153 occurrences de ngrammes.


In [117]:
phrases[:15]

[[('dénomination',), 'NOM', 1120],
 [('dénomination', 'commune'), 'NOM ADJ', 1120],
 [('alfa',), 'NOM', 51],
 [('nom',), 'NOM', 1442],
 [('nom', 'du', 'fabricant'), 'NOM PRP:det NOM', 1042],
 [('fabricant',), 'NOM', 1324],
 [('teneur',), 'NOM', 1035],
 [('indication',), 'NOM', 1789],
 [('recommandation',), 'NOM', 1185],
 [('recommandation', 'de', "l'", 'inesss'), 'NOM PRP DET:ART ADJ', 1014],
 [('recommandation', 'de', "l'", 'inesss', 'modification'),
  'NOM PRP DET:ART ADJ NOM',
  82],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indication'),
  'NOM PRP DET:ART ADJ NOM PRP DET:ART NOM',
  80],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indication',
   'reconnue'),
  'NOM PRP DET:ART ADJ NOM PRP DET:ART NOM VER:pper',
  78],
 [('modification',), 'NOM', 238],
 [('modification', "d'", 'une', 'indication'), 'NOM PRP DET:ART NOM', 126]]

In [118]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('dénomination',), 'NOM', 1120],
 [('dénomination', 'commune'), 'NOM ADJ', 1120],
 [('alfa',), 'NOM', 51],
 [('nom',), 'NOM', 1442],
 [('nom', 'du', 'fabricant'), 'NOM PRP:det NOM', 1042],
 [('fabricant',), 'NOM', 1324],
 [('teneur',), 'NOM', 1035],
 [('indication',), 'NOM', 1789],
 [('recommandation',), 'NOM', 1185],
 [('recommandation', 'de', "l'", 'inesss'), 'NOM PRP DET:ART ADJ', 1014],
 [('recommandation', 'de', "l'", 'inesss', 'modification'),
  'NOM PRP DET:ART ADJ NOM',
  82],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indication'),
  'NOM PRP DET:ART ADJ NOM PRP DET:ART NOM',
  80],
 [('recommandation',
   'de',
   "l'",
   'inesss',
   'modification',
   "d'",
   'une',
   'indication',
   'reconnue'),
  'NOM PRP DET:ART ADJ NOM PRP DET:ART NOM VER:pper',
  78],
 [('modification',), 'NOM', 238],
 [('modification', "d'", 'une', 'indication'), 'NOM PRP DET:ART NOM', 126],
 [('modification', "d'", 'une', 'indication', 'reconnu

In [119]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 8778 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [120]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # Pour prendre les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [121]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [122]:
terms = filter_patterns(phrases)
#terms = phrases

In [123]:
terms

[[('dénomination',), 'NOM', 1120],
 [('dénomination', 'commune'), 'NOM ADJ', 1120],
 [('alfa',), 'NOM', 51],
 [('nom',), 'NOM', 1442],
 [('nom', 'du', 'fabricant'), 'NOM PRP:det NOM', 1042],
 [('fabricant',), 'NOM', 1324],
 [('teneur',), 'NOM', 1035],
 [('indication',), 'NOM', 1789],
 [('recommandation',), 'NOM', 1185],
 [('recommandation', 'de', "l'", 'inesss'), 'NOM PRP DET:ART ADJ', 1014],
 [('recommandation', 'de', "l'", 'inesss', 'modification'),
  'NOM PRP DET:ART ADJ NOM',
  82],
 [('modification',), 'NOM', 238],
 [('modification', "d'", 'une', 'indication'), 'NOM PRP DET:ART NOM', 126],
 [('modification', "d'", 'une', 'indication', 'reconnue'),
  'NOM PRP DET:ART NOM VER:pper',
  122],
 [('modification',
   "d'",
   'une',
   'indication',
   'reconnue',
   'par',
   'le',
   'conseil'),
  'NOM PRP DET:ART NOM VER:pper PRP DET:ART NOM',
  30],
 [('indication',), 'NOM', 1789],
 [('indication', 'reconnue'), 'NOM VER:pper', 607],
 [('indication', 'reconnue', 'par', 'le', 'conseil'

In [124]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 20 % des termes
On avait 672153 ngrammes, on en a maintenant 540103.


In [125]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [126]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [127]:
terms_patterns

[{'Expression': ('dénomination',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 1120},
 {'Expression': ('dénomination', 'commune'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 1120},
 {'Expression': ('alfa',), 'Structure syntaxique': 'NOM', 'Fréquence': 51},
 {'Expression': ('nom',), 'Structure syntaxique': 'NOM', 'Fréquence': 1442},
 {'Expression': ('nom', 'du', 'fabricant'),
  'Structure syntaxique': 'NOM PRP:det NOM',
  'Fréquence': 1042},
 {'Expression': ('fabricant',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 1324},
 {'Expression': ('teneur',), 'Structure syntaxique': 'NOM', 'Fréquence': 1035},
 {'Expression': ('indication',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 1789},
 {'Expression': ('recommandation',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 1185},
 {'Expression': ('recommandation', 'de', "l'", 'inesss'),
  'Structure syntaxique': 'NOM PRP DET:ART ADJ',
  'Fréquence': 1014},
 {'Expression': ('recommandation', 'de', "l'", 'inesss', 'modifica

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [128]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [129]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 540103 ngrammes.


In [130]:
frequencies

FreqDist({('STOP',): 59180, ('de',): 52152, (')',): 27611, ('(',): 27524, ('la',): 21106, ("d'",): 20794, ('.',): 19498, ("l'",): 19334, ('des',): 18295, ('et',): 17645, ...})

In [131]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [132]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.01 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [133]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [134]:
significant_coll


[{'Terme': 'dénomination',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 1120,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('dénomination', 'commune'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence (TF)': 1120,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': 'alfa',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 51,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'nom',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 1442,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('nom', 'du', 'fabricant'),
  'Structure syntaxique': 'NOM PRP:det NOM',
  'Fréquence (TF)': 1042,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': 'fabricant',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 1324,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'teneur',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 1035,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'indication',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 1789,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'recommandation',
  'Str

In [135]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 0 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [136]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value
376102,traitement,NOM,7872,-,-
492831,jamp,NOM,7296,-,-
171436,évaluation,NOM,6785,-,-
196572,médicaments,NOM,5448,-,-
76979,demande,NOM,3662,-,-
...,...,...,...,...,...
388811,niveaux de soins alternatifs,NOM PRP NOM ADJ,16,273.138562,0.0
388813,soins alternatifs,NOM ADJ,16,168.230095,0.0
501746,paralysie,NOM,16,-,-
159470,dystonie,NOM,16,-,-


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [137]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [138]:
max_df = round(0.98 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 98% des documents 
max_df

1994

In [139]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

1967

In [140]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [141]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > 0} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [142]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [143]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,traitement,1218
1,jamp,992
2,évaluation,1396
3,médicaments,1206
4,demande,280
...,...,...
6020,niveaux de soins alternatifs,7
6021,soins alternatifs,7
6022,paralysie,7
6023,dystonie,2


In [144]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [145]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,traitement,NOM,7872,-,-,1218
1,jamp,NOM,7296,-,-,992
2,évaluation,NOM,6785,-,-,1396
3,médicaments,NOM,5448,-,-,1206
4,demande,NOM,3662,-,-,280
...,...,...,...,...,...,...
5858,degré,NOM,16,-,-,14
5859,foam,NOM,16,-,-,8
5860,coque,NOM,16,-,-,22
5861,médicaments du québec,NOM PRP:det NOM,16,88.74282,0.0,13


In [146]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [147]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [148]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [149]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [150]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [151]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [152]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [153]:
df['isMeSHTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,traitement,NOM,7872,-,-,1218,False
1,jamp,NOM,7296,-,-,992,False
2,évaluation,NOM,6785,-,-,1396,False
3,médicaments,NOM,5448,-,-,1206,False
4,demande,NOM,3662,-,-,280,False
...,...,...,...,...,...,...,...
5858,degré,NOM,16,-,-,14,False
5859,foam,NOM,16,-,-,8,False
5860,coque,NOM,16,-,-,22,False
5861,médicaments du québec,NOM PRP:det NOM,16,88.74282,0.0,13,False


In [154]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [155]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [156]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,traitement,NOM,7872,-,-,1218,False
1,jamp,NOM,7296,-,-,992,False
2,évaluation,NOM,6785,-,-,1396,False
3,médicaments,NOM,5448,-,-,1206,False
4,demande,NOM,3662,-,-,280,False
...,...,...,...,...,...,...,...
5858,degré,NOM,16,-,-,14,False
5859,foam,NOM,16,-,-,8,False
5860,coque,NOM,16,-,-,22,False
5861,médicaments du québec,NOM PRP:det NOM,16,88.74282,0.0,13,False


In [157]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,traitement,NOM,7872,-,-,1218,False
1,jamp,NOM,7296,-,-,992,False
2,évaluation,NOM,6785,-,-,1396,False
3,médicaments,NOM,5448,-,-,1206,False
4,demande,NOM,3662,-,-,280,False
...,...,...,...,...,...,...,...
5858,degré,NOM,16,-,-,14,False
5859,foam,NOM,16,-,-,8,False
5860,coque,NOM,16,-,-,22,False
5861,médicaments du québec,NOM PRP:det NOM,16,88.74282,0.0,13,False


In [158]:
df[df['isMeSHTerm'] == 'True']

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
24,bicarbonate de sodium,NOM PRP NOM,1952,-inf,1.0,974,True
138,services sociaux,NOM ADJ,842,-inf,1.0,571,True
170,cancer du poumon,NOM PRP:det NOM,533,-inf,1.0,109,True
234,cancer du poumon non à petites cellules,NOM PRP:det NOM ADV PRP ADJ NOM,383,-inf,1.0,71,True
269,soins palliatifs,NOM ADJ,323,-inf,1.0,106,True
...,...,...,...,...,...,...,...
5964,helicobacter pylori,NOM ADJ,16,375.267628,0.0,6,True
5802,prophylaxie postexposition,NOM NOM,16,255.49229,0.0,8,True
5811,protocole de recherche,NOM PRP NOM,16,218.357688,0.0,8,True
5787,jonction oesogastrique,NOM ADJ,16,343.464164,0.0,6,True


### **Extraction de termes existant dans la taxonomie**

In [159]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,traitement,NOM,7872,-,-,1218,False,False
1,jamp,NOM,7296,-,-,992,False,False
2,évaluation,NOM,6785,-,-,1396,False,False
3,médicaments,NOM,5448,-,-,1206,False,False
4,demande,NOM,3662,-,-,280,False,False
...,...,...,...,...,...,...,...,...
5858,degré,NOM,16,-,-,14,False,False
5859,foam,NOM,16,-,-,8,False,False
5860,coque,NOM,16,-,-,22,False,False
5861,médicaments du québec,NOM PRP:det NOM,16,88.74282,0.0,13,False,False


In [160]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    labels = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

labels

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [161]:
for t in df['Terme'].tolist():
    if t in labels:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,traitement,NOM,7872,-,-,1218,False,True
1,jamp,NOM,7296,-,-,992,False,False
2,évaluation,NOM,6785,-,-,1396,False,False
3,médicaments,NOM,5448,-,-,1206,False,True
4,demande,NOM,3662,-,-,280,False,False
...,...,...,...,...,...,...,...,...
5858,degré,NOM,16,-,-,14,False,False
5859,foam,NOM,16,-,-,8,False,False
5860,coque,NOM,16,-,-,22,False,True
5861,médicaments du québec,NOM PRP:det NOM,16,88.74282,0.0,13,False,False


In [162]:
# Par exemple, on peut extraire les termes MeSH qui ne sont pas dans la taxonomie, mais qui se trouvent dans le corpus d'intérêt
df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()

C:\Users\p1115145\AppData\Local\Temp\ipykernel_9716\4006263597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()


,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
24,bicarbonate de sodium,NOM PRP NOM,1952,-inf,1.0,974,True,False
437,hypertension artérielle,NOM ADJ,200,-inf,1.0,114,True,False
541,médicaments biosimilaires,NOM ADJ,191,-inf,1.0,88,True,False
625,qualité de vie,NOM PRP NOM,157,-inf,1.0,106,True,False
718,point de vue,NOM PRP NOM,132,-inf,1.0,123,True,False
...,...,...,...,...,...,...,...,...
6003,anticorps monoclonal,NOM ADJ,16,310.127298,0.0,16,True,False
5802,prophylaxie postexposition,NOM NOM,16,255.49229,0.0,8,True,False
5811,protocole de recherche,NOM PRP NOM,16,218.357688,0.0,8,True,False
5787,jonction oesogastrique,NOM ADJ,16,343.464164,0.0,6,True,False


In [163]:
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 
df.to_csv(output_path)